In [168]:
#importando bibliotecas
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException    
import json


In [169]:
#definindo a url
url = 'http://www1.dnit.gov.br/editais/consulta/editais2.asp'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
time.sleep(5)

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240


[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/86.0.4240.22/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\rysah\.wdm\drivers\chromedriver\win32\86.0.4240.22]


In [170]:
#clica para mostrar todas as linhas
popup = driver.find_element_by_xpath("//div[@class='dataTables_length']//label//select[@name='tabela_length']//option[text()='Todos']").click()
time.sleep(20)

In [171]:
#seleciono pela tag table com seu id
element = driver.find_element_by_xpath("//div[@class='dataTables_wrapper']//table[@id='tabela']")
html_content = element.get_attribute('outerHTML')

In [184]:
#analisa o documento
soup = BeautifulSoup(html_content,'html.parser')
parsed_table =soup.find(name='table')
#nessa parte faço um for para poder pegar o valor do href e concateno com o inicio da url
data = [['http://www1.dnit.gov.br/editais/consulta/' + td.a['href'] if td.find('a') else 
             ''.join(td.stripped_strings)
             for td in row.find_all('td')]
            for row in parsed_table.find_all('tr')]
#Seleciono as informações que eu preciso
df = pd.DataFrame(data[2:],columns=data[7])
#defino o nome das colunas
df.columns = [ 'SUP_Regional', 'Modalidade', 'Status','Numero','Data_da_Abertura','Objeto','Detalhes']

In [185]:
#crio algumas listas para poder pegar dados de uma outra pagina (do link gerado no for anterior)
listProcesso=[]
listModalidade=[]
listJulgamento=[]
listValor=[]
listDataBase=[]
for row in df.itertuples():
    try:         
        #abro o link para poder pegar os dados da pagina
        urlLink = row.Detalhes
        driver.get(urlLink)            
        elementLink = driver.find_element_by_xpath("//div[@class='panel-body']")
        html_contentLink = elementLink.get_attribute('outerHTML')
        soupLink = BeautifulSoup(html_contentLink,'html.parser')
        #loop para poder pegar todas as tag p dentro da class
        for review in soupLink.find_all("div",class_="panel-body"):
            listProcesso.append(review.find_all('p')[1].getText())
            listModalidade.append(review.find_all('p')[5].getText())
            listJulgamento.append(review.find_all('p')[7].getText())
            listValor.append(review.find_all('p')[9].getText())
            listDataBase.append(review.find_all('p')[11].getText())
    except NoSuchElementException as error: 
        #caso tenha algum erro no link, preencho o valor com vazio, para garantir que as listas tenham o mesmo tamanho do dataframe
        listProcesso.append("")
        listModalidade.append("")
        listJulgamento.append("")
        listValor.append("")
        listDataBase.append("")
        continue
#insiro as colunas no dataframe, para poder preencher com o restante das informações    
df.insert(7,"Processo",listProcesso,True)
df.insert(8,"Modalidade",listModalidade,True)
df.insert(9,"Julgamento",listJulgamento,True)
df.insert(10,"Valor_Global",listValor,True)
df.insert(11,"DataBase",listDataBase,True)


In [186]:
#Ler os dados e salva em um arquivo json
with open('dnitTotal.json', 'w', encoding='utf-8') as jp:
    js = json.dumps(df.to_dict('records'))
    jp.write(js)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:3: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  This is separate from the ipykernel package so we can avoid doing imports until


In [187]:
#gerando um excel com as informações obtidas
df.to_excel('DNITTotal.xlsx', engine='xlsxwriter') 